<a href="https://colab.research.google.com/github/hululuzhu/chinese-ai-writing-share/blob/main/further_finetune_example/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference for T5 model that mimics Li's style
- Download my saved models at Drive
  - [General Version](https://drive.google.com/drive/folders/1ZymaSbOcwlslD5tuUIk_9__C2dUJK_UY?usp=sharingY) and [ericqianli's mimic version](https://drive.google.com/drive/folders/10YyRO3-8MLVziqme1wnpqN5RPyDRuLa-?usp=sharing)
  - 重要：以上文件都存在Google Drive，推荐用Google账号打开，点击`Add to shortcut`，之后在你Drive的主页面`shared with me`看到目录后选择`add shortcut to Drive`，这样可以mount后本地可以操作文件，但要注意路径一致

## Load package and previously trained models

In [ ]:
# Quite install simple T5 package
!pip install -q simplet5 &> /dev/null
!pip install -q chinese-converter > /tmp/na
import chinese_converter  # 繁体到简体需要

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/ML/Models/t5-poem-li-2022branch/simplet5-epoch-5-train-loss-0.3138-val-loss-0.4084
!ls /content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/*

config.json	   special_tokens_map.json  tokenizer_config.json
pytorch_model.bin  spiece.model
/content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/config.json
/content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/inference_mengzi_t5_poem_model.ipynb
/content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/pytorch_model.bin
/content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/special_tokens_map.json
/content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/spiece.model
/content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/tokenizer_config.json


In [ ]:
!mkdir -p my_t5/poem/li
!mkdir -p my_t5/poem/other
# 6 epochs, 10min hours P100 16G
!cp /content/drive/MyDrive/ML/Models/t5-poem-li-2022branch/simplet5-epoch-5-train-loss-0.3138-val-loss-0.4084/* my_t5/poem/li
!cp /content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/* my_t5/poem/other

In [ ]:
import torch
from simplet5 import SimpleT5
from transformers import T5Tokenizer, T5ForConditionalGeneration

class MengziSimpleT5(SimpleT5):
  def __init__(self) -> None:
    super().__init__()
    self.device = torch.device("cuda")

  def load_my_model(self, local_path, use_gpu: bool = True):
    self.tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")
    self.model = T5ForConditionalGeneration.from_pretrained(local_path)

Global seed set to 42


In [ ]:
AUTHOR_PROMPT = "模仿："
TITLE_PROMPT = "作诗："
EOS_TOKEN = '</s>'

general_poem_model = MengziSimpleT5()
general_poem_model.load_my_model(local_path='my_t5/poem/other')
general_poem_model.model = general_poem_model.model.to('cuda')

li_poem_model = MengziSimpleT5()
li_poem_model.load_my_model(local_path='my_t5/poem/li')
li_poem_model.model = li_poem_model.model.to('cuda')

MAX_AUTHOR_CHAR = 4
MAX_TITLE_CHAR = 12
MIN_CONTENT_CHAR = 10
MAX_CONTENT_CHAR = 64

def poem(title_str, opt_author=None, is_input_traditional_chinese=False):
  model = li_poem_model if opt_author == '钱力' else general_poem_model
  model.model = model.model.to('cuda')
  if opt_author:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR] + EOS_TOKEN + AUTHOR_PROMPT + opt_author[:MAX_AUTHOR_CHAR]
  else:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR]
  if is_input_traditional_chinese:
    in_request = chinese_converter.to_simplified(in_request)
  out = model.predict(in_request,
                      max_length=MAX_CONTENT_CHAR)[0].replace(",", "，")
  if is_input_traditional_chinese:
    out = chinese_converter.to_traditional(out)
    print(f"標題： {in_request.replace('</s>', ' ')}\n詩歌： {out}")
  else:
    print(f"标题： {in_request.replace('</s>', ' ')}\n诗歌： {out}")

## Inference now

In [ ]:
for title in ['秋', "百花", '佳人有约', "中秋", "抚琴一首"]:
  # Empty author means general style
  for author in ["", "杜甫", "李白", "李清照", "苏轼", "钱力"]:
    poem(title, author)
  print()

标题： 作诗：秋
诗歌： 西风吹雨过前溪，篱落萧条菊未齐。黄叶满庭秋色老，碧云无路客愁低。
标题： 作诗：秋 模仿：杜甫
诗歌： 西风动高树，落日满长空。白露沾衣湿，黄花带叶红。客愁随暮角，归思逐秋风。未有还家夢，沧浪一钓翁。
标题： 作诗：秋 模仿：李白
诗歌： 秋色满西楼，凉风动客愁。雁来天北去，人去水东流。白露沾衣冷，黄花入鬓秋。何当一樽酒，相与醉高楼。
标题： 作诗：秋 模仿：李清照
诗歌： 西风吹雨过江城，客子愁多酒易醒。黄叶满庭秋色老，碧云无路夕阳青。
标题： 作诗：秋 模仿：苏轼
诗歌： 西风吹雨过江城，黄叶萧萧满院声。秋色已随人意老，客愁还共酒杯清。
标题： 作诗：秋 模仿：钱力
诗歌： 玉管金弦风流罢，奈落烟宵不尽愁。闲来总惹碧山夢，云外苍天化外秋。

标题： 作诗：百花
诗歌： 百花开尽绿阴成，红紫妖红照眼明。谁道东风无意思，一枝春色爲谁荣。
标题： 作诗：百花 模仿：杜甫
诗歌： 百花开尽绿阴成，独有江梅照眼明。莫道春光无别意，只应留得一枝横。
标题： 作诗：百花 模仿：李白
诗歌： 百花如锦树，春色满芳洲。日暖花争发，风轻絮乱流。香飘金谷露，艳拂玉山楼。谁道无情物，年年爲客愁。
标题： 作诗：百花 模仿：李清照
诗歌： 百花如锦水如蓝，春到园林处处堪。谁道东风不相识，一枝开尽绿阴南。
标题： 作诗：百花 模仿：苏轼
诗歌： 百花开尽绿阴成，谁道春风不世情。若使此花无俗韵，世间那得有芳名。
标题： 作诗：百花 模仿：钱力
诗歌： 谁将碧血染春山，幻作芳春烂漫看。无奈东风相逐去，落英狼借到天寒。

标题： 作诗：佳人有约
诗歌： 佳人约我共登台，笑指花前酒半杯。莫道春光无分到，且看红日上楼来。
标题： 作诗：佳人有约 模仿：杜甫
诗歌： 佳人有约到江干，共约寻春入肺肝。红杏绿桃相映发，白苹红蓼不胜寒。花前醉舞春风裏，月下狂歌夜漏残。莫怪相逢不相识，只应清夢在长安。
标题： 作诗：佳人有约 模仿：李白
诗歌： 佳人有约在瑶台，花落花开不待开。莫道春风无分到，且看明月照楼台。
标题： 作诗：佳人有约 模仿：李清照
诗歌： 佳人约我共登台，花下相携醉不回。莫道春归无觅处，桃花依旧笑人来。
标题： 作诗：佳人有约 模仿：苏轼
诗歌： 佳人约我共清欢，笑指花前醉玉盘。莫道春归无觅处，且看红日上栏干。
标题： 作诗：佳人有约 模仿：钱力
诗歌： 佳期无约久，